In [ ]:
# Load Dependencies
import os
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
from IPython.display import Markdown
import featuretools as ft

pd.set_option('display.max_rows', None)

# Load Data
df = pd.read_csv('data/data_raw.csv.gz', compression='gzip')

In [ ]:
# Create a new column for modified country values
df['country_top'] = df['country']
df['state_top'] = df['state']

# Replace missing values with 'Missing'
df.loc[df['country_top'].isna(), 'country_top'] = 'Missing'
df.loc[df['state_top'].isna(), 'state_top'] = 'Missing'

# Get top 10 states and countries
top_countries = df['country_top'].value_counts().nlargest(10).index
top_states = df['state_top'].value_counts().nlargest(10).index

# Replace countries not in the top 10 with 'Other'
df.loc[~df['country_top'].isin(top_countries), 'country_top'] = 'Other'
df.loc[~df['state_top'].isin(top_states), 'state_top'] = 'Other'

# Dummy variables for top countries and missing
df = pd.concat([df, pd.get_dummies(df['country_top'], prefix='country_top')], axis=1)
df = pd.concat([df, pd.get_dummies(df['state_top'], prefix='state_top')], axis=1)

# Filter columns that contain 'country_top' or 'state_top'
dummies = df.filter(like='country_top_').columns.union(df.filter(like='state_top_').columns)

# Dummy variables for industry, revenue, and founded time
df = pd.concat([df, pd.get_dummies(df['industry_grouped'].astype('str').replace({'NaN': 'Missing'}), prefix='industry')], axis=1)
dummies = dummies.union(df.filter(like='industry_').columns)
df = pd.concat([df, pd.get_dummies(df['company_revenue_bucket'].astype('str').replace({'NaN': 'Missing'}), prefix='revenue')], axis=1)
dummies = dummies.union(df.filter(like='revenue_').columns)
df = pd.concat([df, pd.get_dummies(df['dnb_founded_time_grouped'].astype('str').replace({'NaN': 'Missing'}), prefix='founded')], axis=1)
dummies = dummies.union(df.filter(like='founded_').columns)

# Recode True/False/Nan to 1/-1/0 for "boolean" variables to treat missing as its own category
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].replace({1: 1, 0: -1, np.nan: 0})

# Display dummy column counts
display(df[dummies].nunique().to_frame().T)

# Write out data
df.to_csv('data/dummied.csv.gz', compression='gzip', index=False)


In [ ]:
df.info()

In [ ]:
# ydata-profiling of df
if not os.path.exists('profile/ydata_tx.html'):
    profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)
    profile.to_file("profile/ydata_tx.html")

# ydata-profiling of df with is_current_customer = True and False
if not os.path.exists('profile/ydata_is_customer.html'):
    profile = ProfileReport(df[df['is_current_customer'] == True], title='Current Customer', explorative=True)
    profile.to_file("profile/ydata_is_customer.html")
if not os.path.exists('profile/ydata_not_customer.html'):
    profile = ProfileReport(df[df['is_current_customer'] == False], title='NOT Current Customer', explorative=True)
    profile.to_file("profile/ydata_not_customer.html")

# ydata-profiling of df with is_arr_over_12k = True and False
if not os.path.exists('profile/ydata_arr_over_12k.html'):
    profile = ProfileReport(df[df['is_arr_over_12k'] == True], title='ARR over $12k', explorative=True)
    profile.to_file("profile/ydata_arr_over_12k.html")
if not os.path.exists('profile/ydata_arr_under_12k.html'):
    profile = ProfileReport(df[df['is_arr_over_12k'] == False], title='ARR under $12k', explorative=True)
    profile.to_file("profile/ydata_arr_under_12k.html")


In [ ]:
# Select columns for feature engineering
exclude = ['is_current_customer', 'is_self_service', 'is_arr_over_12k', 'country', 'state', 'country_top', 'state_top', 'industry_grouped', 'company_revenue_bucket', 'dnb_founded_time_grouped']
selected = df.drop(columns=exclude)

customers = df[df['is_current_customer'] == True]
correlations = customers.corr(numeric_only=True)['is_arr_over_12k'].abs().sort_values(ascending=False)

# # Display correlation matrix
display(correlations[correlations > 0.08].to_frame().T)
correlations.to_csv('data/correlations.csv')
